In [241]:
import pandas as pd
import numpy
from sklearn.ensemble import RandomForestClassifier as rfc

np.random.seed(1234)

#read in dataset
df = pd.read_csv("data.csv")
df = df.drop(["id","Unnamed: 32"],axis=1)
df = df.replace({'diagnosis': "M"}, 1)
df = df.replace({'diagnosis': "B"}, 0)
labels = df["diagnosis"]
df = df.drop("diagnosis",axis=1)


#values formatted
dfTrn, dfDev, dfTes = numpy.split(df, [int(.15*len(df)), int(.7*len(df))])
DTrn, DDev, DTes = numpy.split(labels, [int(.15*len(labels)), int(.7*len(labels))])

#run model and test
model = rfc()
model = model.fit(dfTrn.values,DTrn)
model_score = model.score(dfDev,DDev)
print("Accuracy(%):",model_score*100)

#pretty accurate

Accuracy(%): 90.7348242812


In [264]:
import pandas as pd
import numpy
import pandas_montecarlo
from scipy.stats import shapiro, kruskal, f_oneway
from sklearn.ensemble import RandomForestClassifier as rfc
from sklearn.neighbors import KNeighborsClassifier as knc
from sklearn.svm import SVC as svc
from sklearn.linear_model import LogisticRegression as lgr
import numpy as np

## RandomForest Classifier with monte carlo simulated training set
np.random.seed(1234)

#df = pd.read_csv("mc_test_data.csv")
df = pd.read_csv("rndf_filt_data.csv")
#print(df.head())
#df = df.drop(["id","Unnamed: 32"],axis=1)
df = df.drop(["Unnamed: 0"],axis=1)
df = df.replace({'diagnosis': "M"}, 1)
df = df.replace({'diagnosis': "B"}, 0)

#split dataset for mc seed and testing

#df_mc, df = numpy.split(df, [int(.7*len(df))])

#split dataset by class
df_1 = pd.read_csv("mc_data_M.csv").drop(["Unnamed: 0"],axis=1)
df_0 = pd.read_csv("mc_data_B.csv").drop(["Unnamed: 0"],axis=1)
#df_1 = df_mc.loc[df_mc.diagnosis==1]
#df_0 = df_mc.loc[df_mc.diagnosis==0]
#df_1 = df_1.drop(["diagnosis"],axis=1)
#df_0 = df_0.drop(["diagnosis"],axis=1)

#simulate class 0 data
mc_sim_df_0 = pd.DataFrame()
mc_sim_df_0['diagnosis']= ['0'] * len(df_0.index)
for col in df_0.columns:
    col_sim = df_0[col].montecarlo(sims = 2, bust = 0, goal = 0).data
    col_sim = col_sim.drop(["original"],axis = 1)
    for col2 in col_sim.columns:
        mc_sim_df_0[col]=col_sim[col2]
        #if(shapiro(mc_sim_df_1[col])[1]>0.05):
            #print(kruskal(mc_sim_df_1[col],df_1[col]))
        #else:
            #print(f_oneway(mc_sim_df_1[col],df_1[col]))

#simulate class 1 data
mc_sim_df_1 = pd.DataFrame()
mc_sim_df_1['diagnosis']= ['1'] * len(df_1.index)
for col in df_1.columns:
    col_sim = df_1[col].montecarlo(sims = 2, bust = 0, goal = 0).data
    col_sim = col_sim.drop(["original"],axis = 1)
    for col2 in col_sim.columns:
        mc_sim_df_1[col]=col_sim[col2]
        #if(shapiro(mc_sim_df_1[col])[1]>0.05):
            #print(kruskal(mc_sim_df_1[col],df_1[col]))
        #else:
            #print(f_oneway(mc_sim_df_1[col],df_1[col]))


#diag = mc_sim_df_1.append(mc_sim_df_0)['diagnosis']
mc_sim_df = mc_sim_df_1.append(mc_sim_df_0)
#shuffling dataframe for good luck
#mc_sim_df = mc_sim_df.sample(frac=1)
#mc_sim_df['diagnosis']=diag
mc_sim_df.head(20)


#values formatted
labels = df["diagnosis"]
df = df.drop("diagnosis",axis=1)
dfDev, dfTes = numpy.split(df, [int(.7*len(df))])
DDev, DTes = numpy.split(labels, [int(.7*len(labels))])

DTrn =  mc_sim_df['diagnosis']
dfTrn = mc_sim_df.drop(['diagnosis'], axis = 1)

#run model and test
#randomforest
model = rfc()
model = model.fit(dfTrn.values,DTrn)
pd = model.predict(dfDev)
hit = 0
for i in range(len(pd)):
    if(int(pd[i])==int(DDev.iloc[i])):
        hit+=1
print("random forest", hit/len(pd))

#knn
model = knc()
model = model.fit(dfTrn.values,DTrn)
pd = model.predict(dfDev)
hit = 0
for i in range(len(pd)):
    if(int(pd[i])==int(DDev.iloc[i])):
        hit+=1
print("knn", hit/len(pd))

#svc
model = svc(kernel="linear")
model = model.fit(dfTrn.values,DTrn)
pd = model.predict(dfDev)
hit = 0
for i in range(len(pd)):
    if(int(pd[i])==int(DDev.iloc[i])):
        hit+=1
print("svc(linear)", hit/len(pd))

#svc
model = svc(kernel="rbf")
model = model.fit(dfTrn.values,DTrn)
pd = model.predict(dfDev)
hit = 0
for i in range(len(pd)):
    if(int(pd[i])==int(DDev.iloc[i])):
        hit+=1
print("svc(rbf)", hit/len(pd))

#logistic regression
model = lgr()
model = model.fit(dfTrn.values,DTrn)
pd = model.predict(dfDev)
hit = 0
for i in range(len(pd)):
    if(int(pd[i])==int(DDev.iloc[i])):
        hit+=1
print("logistic regression", hit/len(pd))

random forest 0.9271356783919598
knn 0.9095477386934674
svc(linear) 0.9221105527638191
svc(rbf) 0.5678391959798995
logistic regression 0.9045226130653267


In [ ]:
#dataset normality

import pandas as pd
import numpy
from sklearn.svm import SVC
from scipy.stats import shapiro

#read in dataset
df = pd.read_csv("data.csv")
df = df.drop(["id","Unnamed: 32"],axis=1)
df = df.replace({'diagnosis': "M"}, 1)
df = df.replace({'diagnosis': "B"}, 0)
labels = df["diagnosis"]
df = df.drop("diagnosis",axis=1)
for col in df.columns:
    print(col.title(), shapiro(df[col]))
        
#dataset is normal




In [248]:
import pandas as pd
import numpy
df1 = pd.read_csv("mc_data_B.csv").drop(["Unnamed: 0"],axis=1)
print(df1.head(5))
df2 = pd.read_csv("mc_data_M.csv").drop(["Unnamed: 0"],axis=1)
print(df2.head(5))
df3 = pd.read_csv("mc_test_data.csv").drop(["Unnamed: 0"],axis=1)
print(df3.head(5))

   area_worst  concave.points_mean  concave.points_worst  perimeter_worst  \
0       630.5             0.031100               0.07283            96.09   
1       314.9             0.020760               0.06227            65.13   
2       545.9             0.029230               0.05013            84.46   
3       242.2             0.005917               0.02564            57.26   
4       582.6             0.027490               0.06548            89.88   

   radius_worst  
0        14.500  
1        10.230  
2        13.300  
3         8.964  
4        13.760  
   area_worst  concave.points_mean  concave.points_worst  perimeter_worst  \
0      2019.0              0.14710                0.2654            184.6   
1      1709.0              0.12790                0.2430            152.5   
2      1575.0              0.10430                0.1625            152.2   
3      1606.0              0.07400                0.1932            153.2   
4       897.0              0.05985          